# **CASE: *CREDIT SCORE***

---

### Instalação de bibliotecas

In [ ]:
# Para cálculo do VIF
install.packages("rms")
# Para cálculo do KS e AUC
install.packages("ROCR")

### Carregamento de bibliotecas

In [ ]:
library(rms)
library(ROCR)

### Leitura da base de dados

In [ ]:
dados_credito <- read.table(file = "Credit_Score.txt",
                            sep = "\t",
                            dec = ".",
                            header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_credito)

### Análise exploratória breve

In [ ]:
# Medidas resumo
summary(dados_credito)

In [ ]:
# Tabela de frequências: safra
table(dados_credito$SAFRA)
prop.table(table(dados_credito$SAFRA))

In [ ]:
# Tabela de frequências: flag de pagamento parcial
table(dados_credito$FLAG_PGTO_PARCIAL_12M)
prop.table(table(dados_credito$FLAG_PGTO_PARCIAL_12M))

In [ ]:
# Tabela de frequências: mau/bom (resposta)
table(dados_credito$RESPOSTA_MAU_BOM)
prop.table(table(dados_credito$RESPOSTA_MAU_BOM))

### Análise bivariada: variável resposta vs. variáveis explicativas e safra

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 20, repr.plot.height = 12)

In [ ]:
# Gráficos de boxplots
par(mfrow = c(3,4), cex = 1)
boxplot(IDADE                        ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Idade")
boxplot(RENDA_MEDIA_MENSAL           ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Renda Média Mensal")
boxplot(TOTAL_INVESTIMENTOS          ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Valor Total em Investimentos")
boxplot(QTDE_CONSULTAS_CREDITO_12M   ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Qtde. de Consultas de Crédito")
boxplot(QTDE_CARTOES                 ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Qtde. de Cartões")
boxplot(QTDE_EMPRESTIMOS_12M         ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Qtde. de Empréstimos")
boxplot(QTDE_CHEQUE_ESPECIAL_12M     ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Qtde. de Cheque Especial")
boxplot(QTDE_PGTOS_EM_ATRASO_12M     ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Qtde. de Pagamentos em Atraso")
boxplot(TOTAL_DIAS_ATRASO_12M        ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Total de Dias de Atraso")
boxplot(VALOR_PGTOS_12M              ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "Valor Total de Pagamentos")
boxplot(PERC_MEDIO_LIMITE_TOMADO_12M ~ RESPOSTA_MAU_BOM, data = dados_credito, col = "darkturquoise", main = "% Médio de Limite Comprometido")

In [ ]:
# Resetando parâmetros gráficos (recomendado apenas no Colab)
options(reset_options)

In [ ]:
# Tabela de frequências: safra versus resposta (somando 100% nas linhas)
table(dados_credito$SAFRA, dados_credito$RESPOSTA_MAU_BOM)
prop.table(table(dados_credito$SAFRA, dados_credito$RESPOSTA_MAU_BOM), 1)

In [ ]:
# Tabela de frequências: flag de pagamento parcial versus resposta (somando 100% nas linhas)
table(dados_credito$FLAG_PGTO_PARCIAL_12M, dados_credito$RESPOSTA_MAU_BOM)
prop.table(table(dados_credito$FLAG_PGTO_PARCIAL_12M, dados_credito$RESPOSTA_MAU_BOM), 1)

### Modelo de regressão logística múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- glm(RESPOSTA_MAU_BOM ~
                     IDADE +
                     RENDA_MEDIA_MENSAL +
                     QTDE_CONSULTAS_CREDITO_12M +
                     QTDE_CARTOES +
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     TOTAL_DIAS_ATRASO_12M +
                     VALOR_PGTOS_12M +
                     PERC_MEDIO_LIMITE_TOMADO_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando PERC_MEDIO_LIMITE_TOMADO_12M
regressao_2 <- glm(RESPOSTA_MAU_BOM ~
                     IDADE +
                     RENDA_MEDIA_MENSAL +
                     QTDE_CONSULTAS_CREDITO_12M +
                     QTDE_CARTOES +
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     TOTAL_DIAS_ATRASO_12M +
                     VALOR_PGTOS_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_2)

In [ ]:
# Ajuste do modelo 3: retirando RENDA_MEDIA_MENSAL
regressao_3 <- glm(RESPOSTA_MAU_BOM ~
                     IDADE +
                     QTDE_CONSULTAS_CREDITO_12M +
                     QTDE_CARTOES +
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     TOTAL_DIAS_ATRASO_12M +
                     VALOR_PGTOS_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_3)

In [ ]:
# Ajuste do modelo 4: retirando QTDE_CONSULTAS_CREDITO_12M
regressao_4 <- glm(RESPOSTA_MAU_BOM ~
                     IDADE +
                     QTDE_CARTOES +
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     TOTAL_DIAS_ATRASO_12M +
                     VALOR_PGTOS_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_4)

In [ ]:
# Ajuste do modelo 5: retirando QTDE_CARTOES
regressao_5 <- glm(RESPOSTA_MAU_BOM ~
                     IDADE +
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     TOTAL_DIAS_ATRASO_12M +
                     VALOR_PGTOS_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_5)

In [ ]:
# Ajuste do modelo 6: retirando IDADE
regressao_6 <- glm(RESPOSTA_MAU_BOM ~
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     TOTAL_DIAS_ATRASO_12M +
                     VALOR_PGTOS_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_6)

In [ ]:
# Ajuste do modelo 7: retirando TOTAL_DIAS_ATRASO_12M
regressao_7 <- glm(RESPOSTA_MAU_BOM ~
                     QTDE_EMPRESTIMOS_12M +
                     QTDE_CHEQUE_ESPECIAL_12M +
                     QTDE_PGTOS_EM_ATRASO_12M +
                     VALOR_PGTOS_12M,
                   family = binomial(link = 'logit'),
                   data = dados_credito)

summary(regressao_7)

In [ ]:
# Avaliação de colinearidade no modelo 7
data.frame(VIF = vif(regressao_7))

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os parâmetros referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_7$coefficients[beta + 1] - 1.96 * coef(summary(regressao_7))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_7$coefficients[beta + 1] + 1.96 * coef(summary(regressao_7))[beta + 1, "Std. Error"])

### Aplicação do modelo e definição de resposta predita

In [ ]:
# Aplicação do modelo na base (criação de uma nova coluna chamada "probabilidade")
dados_credito$probabilidade <- predict(regressao_7,
                                       dados_credito,
                                       type = "response")

In [ ]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- mean(dados_credito$RESPOSTA_MAU_BOM)
ponto_corte

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_credito$predito <- as.factor(ifelse(dados_credito$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho

In [ ]:
# Tabela de classificação
tabela <- table(dados_credito$RESPOSTA_MAU_BOM, dados_credito$predito)

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_credito$probabilidade, dados_credito$RESPOSTA_MAU_BOM)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_credito$probabilidade, dados_credito$RESPOSTA_MAU_BOM)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo cliente, que possui plano pré-pago, renda mensal de 5.000 reais e 30 anos de idade.
# Obs.: os nomes das colunas e padrão de conteúdo devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(QTDE_EMPRESTIMOS_12M     = c(1),
                          QTDE_CHEQUE_ESPECIAL_12M = c(0),
                          QTDE_PGTOS_EM_ATRASO_12M = c(0),
                          VALOR_PGTOS_12M          = c(10000))

In [ ]:
# Aplicação do modelo
novos_dados$PROB_CANCELAR_6M <- predict(regressao_7, novos_dados, type = "response")
View(novos_dados)